Notes:
- Discuss how we want to introduce the error term in the simplest 1st version of our model.
- Is there a more efficient way to record education level.

As a final product of this notebook we wish to obtain the highest value function among the choice specific value functions for the three labor market choices at each admissible state space point in each period of the model.

In [1]:
import pickle
import numpy as np
import math

As a first step, we need to ensure that all arguments we need to supply to the function are available.

We import the model specification parameters and externally defined constants here.

In [2]:
# Execute entire file and make all variables/functions/classes
# available for further use
from ipynb.fs.full.model_spec import (num_periods,
                                      num_choices,
                                      educ_max,
                                      educ_min,
                                      educ_range,
                                      mu,
                                      delta,
                                      optim_paras)

# Import specified definitions only from given notebook
import ipynb.fs
from .defs.shared_constants import MISSING_INT, MISSING_FLOAT

In a final version of soepy, the pyth_create_state_space state space is called before the backward_induction procedure. Here, we import the final output of the pyth_create_state_function.

In [3]:
# Import the final output of pyth_create_state_space, args
file_name = "args_file.pkl"
# Open the file for reading
file_object = open(file_name,'rb')  
# load the object from the file into var args
args = pickle.load(file_object)

In [4]:
# Unpack objects from agrs
states_all, states_number_period, mapping_states_index, max_states_period = args[0], args[1], args[2], args[3]

The individuals in our model, solve their optimization problem by making a labor supply choice every period. They choose the option that is associated with the highest value function. The value function for each of the 3 alternatives is the sum of the current period flow utility of choosing alternative j and a continuation value.

The flow utility includes the current period wage shock, which the individual becomes aware of in the begining of the period and includes her calculations. To obtain an estimate of the continuation value the individual has to integrate out the distribution of the future shocks. In the model implementstion, we perform numerical integration via a Monte Carlo simulation.

As a next step, we generate draws of the know error term distribution.

In [5]:
# Set seed and number of draws for the Monte Carlo integrstion of the error term out of the continuation value
num_draws_emax = 500 # (in final version, to be supplied at initialization)
seed = 635

# Create draws from the standard normal distribution
np.random.seed(seed)
draws_emax_standard = np.random.multivariate_normal(np.zeros(2), np.identity(2), (num_periods, num_draws_emax))

# Transform disturbances to normal distribution with desired covariance matrix
chocks_cov = [optim_paras[14]**2, optim_paras[15]**2] #Form covariances
draws_emax_risk = draws_emax_standard
draws_emax_risk[:,:,0] = draws_emax_standard[:,:,0]*chocks_cov[0]
draws_emax_risk[:,:,1] = draws_emax_standard[:,:,1]*chocks_cov[1]

#Transform by taking the exponent
draws_emax_risk = np.exp(draws_emax_risk)

In [6]:
draws_emax_risk.shape

(10, 500, 2)

In the current formulation, we assume that the wage process is subject to additive measurement error. The disturbances for the part-time and the full-time wage are normally distributed with mean zero. The spesification assumes no serial and also no contemporaneous correlation across the two error terms.

Finally, we need to define additional functions that are called in the backward induction loop.

In [7]:
def calculate_utilities(educ_lev, exp_p, exp_f, optim_paras, draws):
    """Calculate flow utility for all choices by state and period."""
    
    # Initialize container for utilities at state space point and period
    utilities = np.tile(np.nan, num_choices)
    
    # Calculate utilities for the avaibale joices N, P, F
    
    # Non-employment
    utilities[0] = 0
    
    #Part-time employment
    utilities[1] = 18*(np.exp(np.dot(educ_lev, optim_paras[0:3])) + 
                       np.dot(educ_lev, optim_paras[3:6]) *
                       (exp_p * np.dot(educ_lev, optim_paras[6:9]) + exp_f) 
                       * (1 - np.dot(educ_lev, optim_paras[9:12])) + 1
                      + draws[0])**mu/mu + math.exp(optim_paras[12])
    
    # Full-time employment
    utilities[2] = 38*(np.exp(np.dot(educ_lev, optim_paras[0:3])) + 
                       np.dot(educ_lev, optim_paras[3:6]) *
                       (exp_p * np.dot(educ_lev, optim_paras[6:9]) + exp_f) 
                       * (1 - np.dot (educ_lev, optim_paras[9:12])) + 1
                      + draws[1])**mu/mu + math.exp(optim_paras[13])
    
    return utilities

In [8]:
def construct_emax_risk (period,
                         k,
                         educ_lev,
                         educ_years_idx,
                         num_periods,
                         num_draws_emax,
                         draws_emax_period,
                         states_all,
                         mapping_states_index,
                         optim_paras,
                         periods_emax,
):
    """Obtain the maximum of the value fucntion over the available choices 
    via a Monte Carlo simulation integration procedure.
    """
    
    # Initialize container for sum of maximum value functions
    # over all error term draws for the period and state
    emax = 0.0
    
    # Loop over all error term draws
    # for the period and state currently rached by the parent loop
    for i in range(num_draws_emax):
        
        # Extract the error term draws corresponding to
        # period number, state, and loop iteration number, i
        corresponding_draws = draws_emax_period[i, :]
        
        # Extract relevant state space components 
        educ_years, _, exp_p, exp_f = states_all[period, k, :]
        
        # Calculate flow utility at current period, state, and draw
        flow_utilities = calculate_utilities(educ_lev, exp_p, exp_f, optim_paras, corresponding_draws)
        
        # Obtain continuation values for all choices
        # Initialize container for continuation values
        continuation_values = np.tile(MISSING_FLOAT, 3)
        
        if period != (num_periods - 1):
            
            # Choice: Non-employment
            # Create index for extracting the continuation value
            future_idx = mapping_states_index[period + 1, educ_years_idx, 0, exp_p, exp_f]
            # Extract continuation value
            continuation_values[0] = periods_emax[period + 1, future_idx] 

            # Choice: Part-time
            future_idx = mapping_states_index[period + 1, educ_years_idx, 1, exp_p + 1, exp_f]
            continuation_values[1] = periods_emax[period + 1, future_idx]

            # Choice: Full-time
            future_idx = mapping_states_index[period + 1, educ_years_idx, 2, exp_p, exp_f + 1]
            continuation_values[2] = periods_emax[period + 1, future_idx]
        else:
            continuation_value = np.tile(0.0, num_choices)
        
        # Calculate choice specific value functions
        value_functions = flow_utilities + delta*continuation_values
        
        # Obtain highest value function
        maximum = max(value_functions)
        
        # Add to sum over all draws
        emax += maximum
        
        # End loop
    
    # Average over the number of draws
    emax = emax / num_draws_emax
    
    # Thus, we have integrated out the error term
    
    # Output
    return emax

Before we begin the backward iteration procedure, we initialize the container for the final result.

In [9]:
# Initialize container for the final result,
# maximal value function per perdiod and state:
periods_emax = np.tile(MISSING_FLOAT, (num_periods, max_states_period))

We can now start the backward iteration procedure.

In [10]:
# Loop over all periods
for period in range(num_periods - 1, -1, -1):
    
    # Select the random draws for Monte Carlo integration relevant for the period
    draws_emax_period = draws_emax_risk[period, :, :]
    
    # Loop over all admissible state space points
    # for the period currently reached by the parent loop
    for k in range(states_number_period[period]):
        
        # Unpack state space components
        educ_years = states_all[period, k, 0]
        
        # Extract education information
        if (educ_years <= 10):
            educ_lev = [1,0,0]

        elif (educ_years > 10) and (educ_years <= 12):
            educ_lev = [0,1,0]

        else:
            educ_lev = [0,0,1]
            
        educ_years_idx = educ_years - educ_min
            
            
        # Integrate out the error term
        emax = construct_emax_risk (
            period,
            k,
            educ_lev,
            educ_years_idx,
            num_periods,
            num_draws_emax,
            draws_emax_period,
            states_all,
            mapping_states_index,
            optim_paras,
            periods_emax,
        )
        
        # Record output
        periods_emax[period, k] = emax

Export final output:

In [11]:
# Choose a file name
file_name = "periods_emax_file.pkl"

# Open the file for writing
with open(file_name,'wb') as my_file_obj:
    pickle.dump(periods_emax, my_file_obj)  